In [1]:
### Installing the required packages if not already installed
packages = ['numpy', 'pandas', 'warnings', 'sqlite3', 'scipy', 'matplotlib', 'numba']

for package in packages:
    try:
        __import__(package)
    except ImportError:
        %pip install {package}

import numpy as np    # For numerical computing
import pandas as pd   # For data manipulation
import sqlite3        # For connecting to SQL database
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy import stats
from numba import njit

### Setting the random seed for reproducibility
np.random.seed(42)

### Ignoring the warnings
import warnings
warnings.filterwarnings('ignore')

### Pandas display options
pd.options.display.float_format = '{:.4f}'.format

### Getting some fuctions from the other notebook
%run 99_functions.ipynb

In [2]:
### Extracting the data from the database
conn = sqlite3.connect('Data/data.db')

factor_ret_m = pd.read_sql('SELECT * FROM factors_monthly', conn).set_index('Date').drop(columns='rf')
volatility = pd.read_sql('SELECT * FROM volatility', conn).set_index('Date')
volatility_managed_ptf = pd.read_sql('SELECT * FROM volatility_managed_factor_ptf_outofsample', conn).set_index('Date')

conn.close()

### Making sure all the data is in the same format
factor_ret_m.index = pd.to_datetime(factor_ret_m.index)
volatility.index = pd.to_datetime(volatility.index)
volatility_managed_ptf.index = pd.to_datetime(volatility_managed_ptf.index)

### Since we use a window of 120 months, our data will go from 1993-02 and onwards
factor_ret_m = factor_ret_m.loc[volatility_managed_ptf.index]

### Changing the names of the columns of volatility_managed_ptf to vol_factor_name
volatility_managed_ptf.columns = ['vol_' + col for col in volatility_managed_ptf.columns]

### Combining the two datasets
factor_ext = pd.concat([factor_ret_m, volatility_managed_ptf], axis=1)